In [ ]:
'code pour connaitre le classment de la volatilité des actions du CAC40 sur la période'
import yfinance as yf
import pandas as pd
import numpy as np

# Définitions des symboles boursiers des entreprises
symbols = {
    'Sodexo': 'SW.PA',
    'Air Liquide': 'AI.PA',
    'Airbus': 'AIR.PA',
    'Atos': 'ATO.PA',
    'Danone': 'BN.PA',
    'Orange': 'ORA.PA',
    'BNP Paribas': 'BNP.PA',
    'Saint-Gobain': 'SGO.PA',
    'Vinci': 'DG.PA',
    'L\'Oreal': 'OR.PA',
    'LVMH': 'MC.PA',
    'Credit Agricole': 'ACA.PA',
    'Accor': 'AC.PA',
    'Societe Generale': 'GLE.PA',
    'Sanofi': 'SAN.PA',
    'Michelin': 'ML.PA',
    'Vivendi': 'VIV.PA',
    'Bouygues': 'EN.PA',
    'Pernod Ricard': 'RI.PA',
    'Kering': 'KER.PA',
    'Carrefour': 'CA.PA',
    'Veolia': 'VIE.PA',
    'Thales': 'HO.PA',
    'Engie': 'ENGI.PA',
    'Legrand': 'LR.PA',
    'Schneider Electric': 'SU.PA',
    'Capgemini': 'CAP.PA',
    'Worldline': 'WLN.PA'
}

# Période demandée
start_date = '2023-10-01'
end_date = '2024-10-01'

# Fonction pour calculer le rendement annualisé basé sur la moyenne des rendements journaliers
def calculate_annualized_return(prices):
    if len(prices) == 0:
        return None  # Gestion des erreurs si les données sont vides
    daily_returns = prices.pct_change().dropna()
    mean_daily_return = daily_returns.mean()
    annualized_return = mean_daily_return * 255
    return annualized_return

# Fonction pour calculer la volatilité annualisée
def calculate_annualized_volatility(prices):
    if len(prices) == 0:
        return None  # Gestion des erreurs si les données sont vides
    log_returns = np.log(prices / prices.shift(1)).dropna()
    volatility = log_returns.std() * np.sqrt(252)
    return volatility

# Téléchargement des données historiques des actions
data = {}
for company, symbol in symbols.items():
    stock_data = yf.download(symbol, start=start_date, end=end_date)['Adj Close']
    data[company] = stock_data

# Calcul des rendements et volatilités annualisés
results = []
for company, prices in data.items():
    annualized_return = calculate_annualized_return(prices)
    annualized_volatility = calculate_annualized_volatility(prices)
    results.append({
        'Company': company,
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility
    })

# Création d'un DataFrame
df_results = pd.DataFrame(results)

# Trier par volatilité annualisée croissante
df_sorted = df_results.sort_values(by='Annualized Volatility', ascending=True)

# Affichage des résultats triés
print(df_sorted)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

               Company  Annualized Return  Annualized Volatility
4               Danone           0.281899               0.126350
5               Orange           0.018171               0.140314
23               Engie           0.187528               0.152385
15            Michelin           0.275511               0.167527
1          Air Liquide           0.226284               0.177267
8                Vinci           0.069112               0.180738
11     Credit Agricole           0.259493               0.181097
21              Veolia           0.166406               0.186763
0               Sodexo           0.207229               0.191031
17            Bouygues          -0.014859               0.191451
24             Legrand           0.211436               0.203453
16             Vivendi           0.265772               0.206222
20           Carrefour           0.025332               0.211244
9              L'Oreal           0.059301               0.213291
12               Accor   

In [ ]:
'code pour minimiser la var en se basant sur les actifs les moins corrélés'
import yfinance as yf
import pylab as py
import pulp
import pandas as pd

def Lecture(name):
    return py.array(yf.Ticker(name).history(start="2023-10-01", end="2024-10-01").Close)

def get_Prix(Code):
    Prix = [Lecture(Code[i]) for i in range(len(Code))]
    return Prix

def get_Rdt(Code):
    Prix = get_Prix(Code)
    R = [(Prix[i][1:] - Prix[i][:-1]) / Prix[i][:-1] for i in range(len(Code))]
    return py.array(R)

def NotCorr(Code, t): # t pour nombre d'actifs peu corrélés voulus
    Rdts = get_Rdt(Code)
    Rdts = Rdts.T
    Rdts = pd.DataFrame(Rdts, columns=Code)
    Mat_Corr = Rdts.corr()

    # Calculer la moyenne des corrélations pour chaque actif (sans lui-même)
    Mean_Corr = Mat_Corr.apply(lambda x: (x.sum() - 1) / (len(x) - 1), axis=1)
    # Trouver les actifs avec les plus faibles moyennes de corrélation
    Least_Corr = Mean_Corr.nsmallest(t).index.tolist()

    return Least_Corr

def Rdt_Cov(Code):
    Rdts = get_Rdt(Code)

    Rdt_Mean = [py.mean(Rdts[i]) for i in range(len(Rdts))]
    Mat_Cov = py.cov(Rdts)

    return Rdt_Mean, Mat_Cov

def simulation(R, V, N, t):
    L = py.linalg.cholesky(V).T
    Vect_Actifs = [R + py.dot(py.normal(0, 1, t), L) for i in range(N)]

    return Vect_Actifs

def Optimisation(Rdt_Mean, Mat_Cov, t, epsilon, N, C, temp):
    # Simulation des rendements
    r = simulation(Rdt_Mean, Mat_Cov, N, t)

    # Variables de décision
    gamma = pulp.LpVariable('gamma')
    y = [pulp.LpVariable("y" + str(i), cat='Binary') for i in range(N)]
    x = [pulp.LpVariable("x" + str(i)) for i in range(t)]

    # Définition du problème
    prob = pulp.LpProblem("Minimisation_du_gamma", pulp.LpMinimize)

    # Fonction à optimiser
    prob += gamma

    # Contraintes
    for j in range(N):
        prob += -pulp.lpSum(r[j][i] * x[i] for i in range(t)) <= gamma + C * y[j]

    prob += pulp.lpSum(y) <= int(epsilon * N)
    prob += pulp.lpSum(x) == 1

    if temp == 1:
        prob += x[1] == 0

    # Résolution et affichage
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    print(pulp.LpStatus[prob.status])

    Pond = []
    for i in range(t):
        Pond.append(pulp.value(x[i]))

    Sol_gamma = pulp.value(gamma)

    Sol_y=[]
    for i in range(100):
        Sol_y.append(pulp.value(y[i]))

    print(f"Nombre de pires scénarios inclus dans les pertes (sur 100): {int(sum(Sol_y))}")

    return py.array(Pond), Sol_gamma

def Affichage():
    sd = []
    for i in range(len(Small_Corr_Code)):
        for j in range(len(Small_Corr_Code)):
            if i == j:
                sd.append(py.sqrt(Mat_Cov[i, j]))
    sd = py.array(sd)
    print(f"\n{'Symbole boursier':<20} {'Rendements Moyen':<20} {'Volatilité':<20} {'Pondérations':<10}")
    for i in range(len(Small_Corr_Code)):
        print(f"{Small_Corr_Code[i]:<20} {Rdt_Mean[i]:<20.6f} {sd[i]:<20.6f} {Pond[i]:<10.6f}")

    print(f"\ngamma : {gamma}")
    print(f"\nRendement quotidien espéré du portefeuille : {sum(Rdt_Mean[i] * Pond[i] for i in range(len(Small_Corr_Code))):.6f}")

    print(f"\nVolatilité journalier du portefeuille : {py.sqrt(py.dot(Pond.T, py.dot(Mat_Cov, Pond))):.6f}")

# Code des actifs
Code = [
    'SW.PA', 'AI.PA', 'AIR.PA', 'ATO.PA', 'BN.PA', 'ORA.PA', 'BNP.PA',
    'SGO.PA', 'DG.PA', 'OR.PA', 'MC.PA', 'ACA.PA', 'AC.PA', 'GLE.PA',
    'SAN.PA', 'ML.PA', 'VIV.PA', 'EN.PA', 'RI.PA', 'KER.PA', 'CA.PA',
    'VIE.PA', 'HO.PA', 'ENGI.PA', 'LR.PA', 'SU.PA', 'CAP.PA', 'WLN.PA'
]

Small_Corr_Code = NotCorr(Code, 4)

Rdt_Mean, Mat_Cov = Rdt_Cov(Small_Corr_Code)

for i in range(2):
    print(f"\nTableau {i}")
    Pond, gamma = Optimisation(Rdt_Mean, Mat_Cov, len(Small_Corr_Code), 0.05, 100, 100, i)
    Affichage()



ModuleNotFoundError: No module named 'pulp'

In [ ]:
'code pour minimiser la cvar en se basant sur les actifs au meilleur ratio de Sharpe ou les moins corrélés'

import yfinance as yf
import pylab as py
import pulp
import pandas as pd
import numpy as np  # Importation de numpy

def Lecture(name):
    return py.array(yf.Ticker(name).history(start="2023-10-01", end="2024-10-01").Close)

def get_Prix(Code):
    Prix = [Lecture(Code[i]) for i in range(len(Code))]
    return Prix

def get_Rdt(Code):
    Prix = get_Prix(Code)
    R = [(Prix[i][1:] - Prix[i][:-1]) / Prix[i][:-1] for i in range(len(Code))]
    return py.array(R)

# Fonction pour sélectionner les t actifs les moins corrélés
def NotCorr(Code, t):
    Rdts = get_Rdt(Code)
    Rdts = Rdts.T
    Rdts = pd.DataFrame(Rdts, columns=Code)
    Mat_Corr = Rdts.corr()

    # Calculer la moyenne des corrélations pour chaque actif (sans lui-même)
    Mean_Corr = Mat_Corr.apply(lambda x: (x.sum() - 1) / (len(x) - 1), axis=1)
    # Trouver les actifs avec les plus faibles moyennes de corrélation
    Least_Corr = Mean_Corr.nsmallest(t).index.tolist()

    return Least_Corr

# Fonction pour sélectionner les t actifs ayant le meilleur ratio de Sharpe
def ratio_sharpe(Code, t, risk_free_rate=0):
    Rdts = get_Rdt(Code)

    # Calculer les rendements moyens et la volatilité
    Rdt_Mean = [py.mean(Rdts[i]) for i in range(len(Rdts))]
    Rdt_Std = [py.std(Rdts[i]) for i in range(len(Rdts))]  # Volatilité (écart-type des rendements)

    # Calculer le ratio de Sharpe pour chaque actif
    Sharpe_Ratio = [(Rdt_Mean[i] - risk_free_rate) / Rdt_Std[i] if Rdt_Std[i] != 0 else 0 for i in range(len(Rdt_Mean))]

    # Créer un DataFrame pour stocker les informations
    data = {'Ticker': Code, 'Rdt_Mean': Rdt_Mean, 'Volatility': Rdt_Std, 'Sharpe_Ratio': Sharpe_Ratio}
    df = pd.DataFrame(data)

    # Trier par ratio de Sharpe décroissant et sélectionner les t meilleurs
    top_t = df.sort_values(by='Sharpe_Ratio', ascending=False).head(t)

    return top_t['Ticker'].tolist()

# Fonction pour calculer les rendements moyens et la matrice de covariance
def Rdt_Cov(Code):
    Rdts = get_Rdt(Code)
    Rdt_Mean = [py.mean(Rdts[i]) for i in range(len(Rdts))]
    Mat_Cov = py.cov(Rdts)
    return Rdt_Mean, Mat_Cov

# Fonction de simulation de rendements
def simulation(R, V, N, t):
    L = np.linalg.cholesky(V).T
    Vect_Actifs = [R + np.dot(np.random.normal(0, 1, t), L) for i in range(N)]  # Correction ici
    return Vect_Actifs

# Fonction d'optimisation VaR et CVaR
def Optimisation_VaR_CVaR(Rdt_Mean, Mat_Cov, t, epsilon, N, C, alpha=0.95):
    r = simulation(Rdt_Mean, Mat_Cov, N, t)

    # Variables de décision
    gamma = pulp.LpVariable('gamma')  # VaR
    cvar = pulp.LpVariable('cvar')    # CVaR
    y = [pulp.LpVariable(f'y{i}', cat='Binary') for i in range(N)]
    w = [pulp.LpVariable(f'w{i}', lowBound=0) for i in range(t)]  # Pondérations non négatives

    # Définition du problème de minimisation de la CVaR
    prob = pulp.LpProblem("Minimisation_CVaR", pulp.LpMinimize)

    # Fonction à minimiser : CVaR
    prob += cvar

    # Contraintes pour la VaR et la CVaR
    for j in range(N):
        prob += -pulp.lpSum(r[j][i] * w[i] for i in range(t)) <= gamma + C * y[j]  # VaR constraint
        prob += -pulp.lpSum(r[j][i] * w[i] for i in range(t)) <= cvar  # CVaR constraint

    prob += pulp.lpSum(y) <= int(epsilon * N)  # Nombre de scénarios où les pertes dépassent la VaR
    prob += pulp.lpSum(w) == 1  # La somme des poids doit être égale à 1
    prob += cvar >= gamma + (1 / (1 - alpha)) * (pulp.lpSum(y[j] for j in range(N)) / N)  # Condition de la CVaR

    # Résolution
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    print(pulp.LpStatus[prob.status])

    Pond = [pulp.value(w[i]) for i in range(t)]
    Sol_gamma = pulp.value(gamma)
    Sol_cvar = pulp.value(cvar)

    return np.array(Pond), Sol_cvar  # Ne plus retourner gamma

# Calcul du VaRSR
def calculate_VaRSR(Pond, Rdt_Mean, Mat_Cov, gamma, risk_free_rate=0):
    portfolio_return = sum(Pond[i] * Rdt_Mean[i] for i in range(len(Pond)))
    portfolio_volatility = np.sqrt(np.dot(Pond.T, np.dot(Mat_Cov, Pond)))
    return (portfolio_return - risk_free_rate) / portfolio_volatility - gamma

# Fonction d'affichage des résultats incluant la volatilité journalière de chaque actif, la CVaR et le VaRSR
def Affichage_CVaR_VaRSR(Pond, cvar, VaRSR, Small_Corr_Code, Rdt_Mean, Mat_Cov):
    print(f"\n{'Symbole boursier':<20} {'Rendements Moyen':<20} {'Volatilité Journalière':<20} {'Pondérations':<10}")
    for i in range(len(Small_Corr_Code)):
        volatilité = np.sqrt(Mat_Cov[i][i])  # La volatilité journalière est l'écart-type de la diagonale de la matrice de covariance
        print(f"{Small_Corr_Code[i]:<20} {Rdt_Mean[i]:<20.6f} {volatilité:<20.6f} {Pond[i]:<10.2f}")

    print(f"\nCVaR : {cvar}")
    print(f"Rendement quotidien espéré du portefeuille : {sum(Rdt_Mean[i] * Pond[i] for i in range(len(Small_Corr_Code))):.6f}")
    print(f"Volatilité journalière du portefeuille : {np.sqrt(np.dot(Pond.T, np.dot(Mat_Cov, Pond))):.6f}")
    print(f"VaRSR : {VaRSR:.6f}")

# Code des actifs
Code = [
    'SW.PA', 'AI.PA', 'AIR.PA', 'BN.PA', 'ORA.PA', 'BNP.PA','SGO.PA', 'DG.PA', 'OR.PA', 'MC.PA', 'ACA.PA', 'AC.PA', 'GLE.PA',
    'SAN.PA', 'ML.PA', 'VIV.PA', 'EN.PA', 'RI.PA', 'KER.PA', 'CA.PA','ATO.PA',
    'VIE.PA', 'HO.PA', 'ENGI.PA', 'LR.PA', 'SU.PA', 'CAP.PA', 'WLN.PA'
]

# Permettre à l'utilisateur de choisir entre les actifs les moins corrélés ou ceux avec le meilleur ratio de Sharpe
choix = input("Sélectionnez les actifs basés sur (1) la corrélation ou (2) le ratio de Sharpe : ")
t = int(input("Combien d'actifs voulez-vous sélectionner ? "))

if choix == '1':
    Small_Corr_Code = NotCorr(Code, t)  # Sélectionne les t actifs les moins corrélés
elif choix == '2':
    Small_Corr_Code = ratio_sharpe(Code, t)  # Sélectionne les t actifs avec le meilleur ratio de Sharpe
else:
    print("Choix invalide")
    exit()

# Calcul des rendements moyens et de la matrice de covariance
Rdt_Mean, Mat_Cov = Rdt_Cov(Small_Corr_Code)

# Optimisation uniquement pour la CVaR
Pond, cvar = Optimisation_VaR_CVaR(Rdt_Mean, Mat_Cov, len(Small_Corr_Code), 0.05, 1000, 1000)

# Calcul du VaRSR
VaRSR = calculate_VaRSR(Pond, Rdt_Mean, Mat_Cov, cvar)

# Affichage des résultats incluant noms des actifs, pondérations, CVaR, VaRSR et volatilité journalière
Affichage_CVaR_VaRSR(Pond, cvar, VaRSR, Small_Corr_Code, Rdt_Mean, Mat_Cov)



Sélectionnez les actifs basés sur (1) la corrélation ou (2) le ratio de Sharpe : 1
Combien d'actifs voulez-vous sélectionner ? 4
Optimal

Symbole boursier     Rendements Moyen     Volatilité Journalière Pondérations
ORA.PA               0.000071             0.008806             0.46      
ATO.PA               -0.005620            0.079297             0.02      
SAN.PA               0.000344             0.016257             0.14      
BN.PA                0.001105             0.007974             0.39      

CVaR : 0.016971916
Rendement quotidien espéré du portefeuille : 0.000401
Volatilité journalière du portefeuille : 0.006542
VaRSR : 0.044331
